In [63]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db

from lsst.sims.maf.utils.mafUtils import radec2pix
from lsst.sims.maf.utils import m52snr, astrom_precision, sigma_slope


In [64]:
sigma_slope?

In [65]:
# load opsim database
dbpath_v15 = "/home/idies/workspace/lsst_cadence/FBS_1.5/"  # path to all opsim databases

dbpath_v17 = "/home/idies/workspace/lsst_cadence/FBS_1.7/"

dbpath_v171 = "/home/idies/workspace/lsst_cadence/FBS_1.7.1/"


# output directory
#dataRawDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/tGaps/'

outDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/resultsDb/'
resultsDb = db.ResultsDb(outDir=outDir)



In [66]:
# get the name of all opsim dbs 
import glob

workpath = os.getcwd()
#workpath = '/home/idies/workspace/Storage/lixl/persistent/LSST_OpSim/unknowns'

os.chdir(dbpath_v15)  # change to opsim database directory
dblist_v15 = glob.glob('*.db') 

os.chdir(dbpath_v17)  # change to opsim database directory
dblist_v17 = glob.glob('*.db') 

os.chdir(dbpath_v171)  # change to opsim database directory
dblist_v171 = glob.glob('*.db') 

os.chdir(workpath) # change back to work directory

dblist_v15.sort()
dblist_v17.sort()
dblist_v171.sort()



### get dataSlice 

In [68]:
class getDataMetric(metrics.BaseMetric):
    def __init__(self, colname, **kwargs):
        """colname: str or list"""
        self.colname = colname
        super().__init__(col=colname,**kwargs)
        
    def run(self, dataSlice, slicePoint=None):
        result = dataSlice  # method used to calculate colume data, return a values for each slicepoint
        dataSlice.sort(order='observationStartMJD')
        return result
    
    def reduce_result(self, metric):
        return np.sum(metric['night'])


In [69]:
opsdb=db.OpsimDatabase( dbpath_v17 + dblist_v17[1] )

In [70]:
opsdb.get_column_names()

{'Proposal': ['proposalId', 'proposalName', 'proposalType'],
 'SummaryAllProps': ['observationId',
  'fieldRA',
  'fieldDec',
  'observationStartMJD',
  'flush_by_mjd',
  'visitExposureTime',
  'filter',
  'rotSkyPos',
  'numExposures',
  'airmass',
  'seeingFwhm500',
  'seeingFwhmEff',
  'seeingFwhmGeom',
  'skyBrightness',
  'night',
  'slewTime',
  'visitTime',
  'slewDistance',
  'fiveSigmaDepth',
  'altitude',
  'azimuth',
  'paraAngle',
  'cloud',
  'moonAlt',
  'sunAlt',
  'note',
  'fieldId',
  'proposalId',
  'block_id',
  'observationStartLST',
  'rotTelPos',
  'moonAz',
  'sunAz',
  'sunRA',
  'sunDec',
  'moonRA',
  'moonDec',
  'moonDistance',
  'solarElong',
  'moonPhase',
  'cummTelAz'],
 'info': ['index', 'Parameter', 'Value']}

In [71]:
# metric, slicer, constraint
metric = getDataMetric(colname = [ 'observationStartMJD', 'night', 'fieldRA', 'fieldDec', 
                                   'filter', 'fiveSigmaDepth',
                                   'seeingFwhmGeom'])
#slicer = slicers.UniSlicer()

ra = np.array([34.39339593])
dec = np.array([-5.09032894])
slicer = slicers.UserPointsSlicer(ra,dec)

sqlconstraint = 'night<740'

# bundle
metricSky = metricBundles.MetricBundle(metric, slicer, sqlconstraint)

# group bundle
bundleDict = {'metricSky':metricSky}
group = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir = outDir, resultsDb=resultsDb)

# run
group.runAll()
#group.plotAll(closefigs = False)

Querying database SummaryAllProps with constraint night<740 for columns ['seeingFwhmGeom', 'fiveSigmaDepth', 'observationStartMJD', 'fieldRA', 'fieldDec', 'filter', 'night']
Found 403832 visits
Running:  ['metricSky']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [72]:
dataSlice = metricSky.metricValues.data[0]

In [73]:
pd.DataFrame.from_records(dataSlice)

,seeingFwhmGeom,fiveSigmaDepth,observationStartMJD,fieldRA,fieldDec,filter,night
0,0.789575,22.673869,59865.247108,35.125884,-6.070768,z,12
1,0.690779,22.190989,59865.263273,35.125884,-6.070768,y,12
2,0.692852,22.151354,59865.279172,33.306884,-4.968476,y,12
3,0.669194,22.168417,59865.294027,33.306884,-4.968476,y,12
4,0.848910,22.918077,59868.157452,35.431386,-4.190199,z,15
5,0.847877,22.920179,59868.157869,35.431386,-4.190199,z,15
6,0.847328,22.921536,59868.158286,35.431386,-4.190199,z,15
7,0.846783,22.922886,59868.158702,35.431386,-4.190199,z,15
8,0.846243,22.924231,59868.159119,35.431386,-4.190199,z,15
9,0.845707,22.925570,59868.159536,35.431386,-4.190199,z,15


In [53]:
class LSPMmetric(metrics.BaseMetric): 
        def __init__(self, metricName='LSPMmetric',f='g',surveyduration=10,snr_lim=5,mag_lim=[17,25],percentiles=[2.5,97.5,50], 
                    U=np.arange(-100,100,25),V=np.arange(-100,100,25),W=np.arange(-100,100,25),unusual='uniform',m5Col='fiveSigmaDepth',  
                    mjdCol='observationStartMJD',filterCol='filter', seeingCol='seeingFwhmGeom', nexp= 1,gap_selection = False,dataout=False, 
                    **kwargs):
            '''
        mjdCol= MJD observations column name from Opsim database      (DEFAULT = observationStartMJD) 
        m5Col= Magnitude limit column name from Opsim database      (DEFAULT = fiveSigmaDepth)
        filterCol= Filters column name from Opsim database      (DEFAULT = filter)
        seeingCol = "Geometrical" full-width at half maximum.  (DEFAULT = seeingFwhmGeom)
        f = filter for the observation
        snr_lim = threshold for the signal to noise ratio   
        percentiles = percentile limits to separate known motion from the anomalous one. 
        
            '''
            self.mjdCol = mjdCol 
            self.m5Col = m5Col 
            self.seeingCol = seeingCol 
            self.filterCol= filterCol 
            self.surveyduration =surveyduration 
            self.percentiles = percentiles
            self.snr_lim = snr_lim 
            self.mag_lim = mag_lim
            self.f = f 
            self.nexp = nexp 
            self.U=U 
            self.V=V 
            self.W=W 
            self.gap_selection = gap_selection
            self.dataout = dataout 
             # to have as output all the simulated observed data set dataout=True, otherwise the relative error for  
             # each helpix is estimated 
            sim = pd.read_csv('hyperstar_uniform.csv', usecols=['MAG','MODE','d','PM','PM_out'])
            self.simobj = sim
            if self.dataout: 
                super(LSPMmetric, self).__init__(col=[self.mjdCol,self.filterCol, self.m5Col,self.seeingCol, 'night' ],metricDtype='object', units='', metricName=metricName, 
                                                      **kwargs) 
            else: 
                super(LSPMmetric, self).__init__(col=[self.mjdCol,self.filterCol,  self.m5Col,self.seeingCol, 'night'], 
                                                            units='Proper Motion relative error', metricName=metricName, 
                                                             **kwargs) 
          
            np.seterr(over='ignore',invalid='ignore')
         # typical velocity distribution from litterature (Binney et Tremain- Galactic Dynamics)
            
                         
        def run(self, dataSlice, slicePoint=None): 
            np.random.seed(2500) 
            obs = np.where((dataSlice['filter']==self.f) & (dataSlice[self.mjdCol]<min(dataSlice[self.mjdCol])+365*self.surveyduration)) 
            d = self.simobj['d']
            M = np.array(self.simobj['MAG'])
            mu = np.array(self.simobj['PM'])
            muf = np.array(self.simobj['PM_out'])
            mjd = dataSlice[self.mjdCol][obs]
            if len(dataSlice[self.m5Col][obs])>2: 
                
                # select objects above the limit magnitude threshold 
                snr = m52snr(M[:, np.newaxis],dataSlice[self.m5Col][obs])
                row, col =np.where(snr>self.snr_lim)
                precis = astrom_precision(dataSlice[self.seeingCol][obs], snr[row,:] )
                sigmapm=sigma_slope(dataSlice[self.mjdCol][obs], precis)*365.25*1e3

                #select the objects which displacement can be detected
                if self.gap_selection:
                      Times = np.sort(mjd)
                      dt = np.array(list(combinations(Times,2)))
                      DeltaTs = np.absolute(np.subtract(dt[:,0],dt[:,1]))            
                      DeltaTs = np.unique(DeltaTs)
                      if np.size(DeltaTs)>0:
                                 dt_pm = 0.05*np.amin(dataSlice[self.seeingCol])/muf[np.unique(row)]
                                 selection = np.where((dt_pm>DeltaTs[0]) & (dt_pm<DeltaTs[-1]))
                else:
                      selection = np.unique(row)

                if np.size(selection)>0:
                        print('calculate pm...')
                        pa= np.random.uniform(0,2*np.pi,len(mu[selection]))
                        pm_alpha, pm_delta = mu[selection]*np.sin(pa), mu[selection]*np.cos(pa)
                        pm_un_alpha, pm_un_delta = muf[selection]*np.sin(pa), muf[selection]*np.cos(pa)                    
                        p_min,p_max,p_mean = self.percentiles[0],self.percentiles[1],self.percentiles[2]     
                        mu = mu[selection]
                        muf = muf[selection]
                        sigmapm = sigmapm[selection]
                        Pm = norm.pdf(mu,scale=sigmapm)
                        L = pd.DataFrame([norm.pdf(mu[np.where(position[selection]==p)]) for p in ['H', 'D','B']]).sum(axis=0)
                        L_out = pd.DataFrame([norm.pdf(muf[np.where(position[selection]==p)]) for p in ['H', 'D','B']]).sum(axis=0)
                        Pm_out = norm.pdf(muf,scale=sigmapm)
                        l_frac = np.log(signal.fftconvolve(L_out,Pm_out))-np.log(signal.fftconvolve(L,Pm))                      
                        unusual= np.shape(np.where(l_frac[~np.isnan(l_frac)]>0))[1]
                        res= np.size(unusual)/np.size(selection)   
                        

                        fieldRA = np.mean(dataSlice['fieldRA']) 
                        fieldDec = np.mean(dataSlice['fieldDec'])
                        #dic = {'detected': res,
                        #        'pixID': radec2pix(nside=16, ra=np.radians(fieldRA), dec=np.radians(fieldDec))}  
                        dic= {'detected': res,
                              'pixID': radec2pix(nside=16, ra=np.radians(fieldRA), dec=np.radians(fieldDec)),
                              'PM': pd.DataFrame({'pm_alpha':pm_alpha, 'pm_delta':pm_delta}),
                              'PM_un': pd.DataFrame({'pm_alpha':pm_un_alpha, 'pm_delta':pm_un_delta})}
                                #'PM': np.array(mu)[selection], 
                                #'PM_OUT': pm}
                        if self.dataout:
                            return dic 
                        else: 
                            return res


In [56]:
def run(self, dataSlice, slicePoint=None): 
    np.random.seed(2500) 
    obs = np.where((dataSlice['filter']==self.f) & (dataSlice[self.mjdCol]<min(dataSlice[self.mjdCol])+365*self.surveyduration)) 
    d = self.simobj['d']
    M = np.array(self.simobj['MAG'])
    mu = np.array(self.simobj['PM'])
    muf = np.array(self.simobj['PM_out'])
    mjd = dataSlice[self.mjdCol][obs]
    if len(dataSlice[self.m5Col][obs])>2: 
        
        # select objects above the limit magnitude threshold 
        snr = m52snr(M[:, np.newaxis],dataSlice[self.m5Col][obs])
        row, col =np.where(snr>self.snr_lim)
        precis = astrom_precision(dataSlice[self.seeingCol][obs], snr[row,:] )
        sigmapm=sigma_slope(dataSlice[self.mjdCol][obs], precis)*365.25*1e3
        
        print(sigmapm)
        #select the objects which displacement can be detected
        if self.gap_selection:
              Times = np.sort(mjd)
              dt = np.array(list(combinations(Times,2)))
              DeltaTs = np.absolute(np.subtract(dt[:,0],dt[:,1]))            
              DeltaTs = np.unique(DeltaTs)
              if np.size(DeltaTs)>0:
                         dt_pm = 0.05*np.amin(dataSlice[self.seeingCol])/muf[np.unique(row)]
                         selection = np.where((dt_pm>DeltaTs[0]) & (dt_pm<DeltaTs[-1]))
        else:
              selection = np.unique(row)

        if np.size(selection)>0:
                print('calculate pm...')
                pa= np.random.uniform(0,2*np.pi,len(mu[selection]))
                pm_alpha, pm_delta = mu[selection]*np.sin(pa), mu[selection]*np.cos(pa)
                pm_un_alpha, pm_un_delta = muf[selection]*np.sin(pa), muf[selection]*np.cos(pa)                    
                p_min,p_max,p_mean = self.percentiles[0],self.percentiles[1],self.percentiles[2]     
                mu = mu[selection]
                muf = muf[selection]
                sigmapm = sigmapm[selection]
                Pm = norm.pdf(mu,scale=sigmapm)
                L = pd.DataFrame([norm.pdf(mu[np.where(position[selection]==p)]) for p in ['H', 'D','B']]).sum(axis=0)
                L_out = pd.DataFrame([norm.pdf(muf[np.where(position[selection]==p)]) for p in ['H', 'D','B']]).sum(axis=0)
                Pm_out = norm.pdf(muf,scale=sigmapm)
                l_frac = np.log(signal.fftconvolve(L_out,Pm_out))-np.log(signal.fftconvolve(L,Pm))                      
                unusual= np.shape(np.where(l_frac[~np.isnan(l_frac)]>0))[1]
                res= np.size(unusual)/np.size(selection)   
                

                fieldRA = np.mean(dataSlice['fieldRA']) 
                fieldDec = np.mean(dataSlice['fieldDec'])
                #dic = {'detected': res,
                #        'pixID': radec2pix(nside=16, ra=np.radians(fieldRA), dec=np.radians(fieldDec))}  
                dic= {'detected': res,
                      'pixID': radec2pix(nside=16, ra=np.radians(fieldRA), dec=np.radians(fieldDec)),
                      'PM': pd.DataFrame({'pm_alpha':pm_alpha, 'pm_delta':pm_delta}),
                      'PM_un': pd.DataFrame({'pm_alpha':pm_un_alpha, 'pm_delta':pm_un_delta})}
                        #'PM': np.array(mu)[selection], 
                        #'PM_OUT': pm}
                if self.dataout:
                    return dic 
                else: 
                    return res



In [59]:
lspm_metric.simobj

,MAG,MODE,d,PM,PM_out
0,20.511607,['H'],114695.522467,-0.000046,0.000166
1,20.739662,['D'],117145.627262,0.000090,-0.000024
2,19.990342,['H'],76283.568149,0.000069,0.000089
3,20.994390,['D'],121126.877927,0.000043,-0.000010
4,23.913523,['D'],102637.159857,-0.000103,0.000021
5,24.191637,['D'],64849.797052,0.000243,-0.000057
6,20.718503,['D'],46105.275405,0.000114,-0.000022
7,22.929386,['D'],99227.807801,-0.000053,0.000141
8,18.280863,['H'],73861.187151,0.000071,-0.000205
9,23.283329,['H'],138128.904792,0.000038,-0.000073


In [54]:
lspm_metric = LSPMmetric()

In [57]:
run(lspm_metric, dataSlice)

2.500303080209164e-05
calculate pm...


IndexError: invalid index to scalar variable.

In [55]:
res = lspm_metric.run(dataSlice=dataSlice)



calculate pm...


IndexError: invalid index to scalar variable.